In [1]:
import pandas as pd
from redfin import Redfin
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=1)

client = Redfin()

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Addresses_cleaned.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID
0,38204500060410000,RES,2326,LUAU ST,MESQUITE (DALLAS CO),751503826,38204500060410000
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE (DALLAS CO),751491534,38137500370290000
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500060160000
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500050250000
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE (DALLAS CO),75181,60259500060230000


In [2]:
# Remove unnesessary (DALLAS CO) from PROPERTY_CITY series
df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()

# Add a column with the full address
df['FullAddress'] = df['STREET_NUM'].astype(str) + ' ' + df['FULL_STREET_NAME'] + ' ' + df['PROPERTY_CITY'] + ' ' + 'TX' + ' ' + df['PROPERTY_ZIPCODE'].astype(str).str[:5]

df = df.drop_duplicates(keep=False, subset=["FullAddress"])

df.head()

C:\Users\kayle\AppData\Local\Temp\ipykernel_17520\1303363462.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID,FullAddress
0,38204500060410000,RES,2326,LUAU ST,MESQUITE,751503826,38204500060410000,2326 LUAU ST MESQUITE TX 75150
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE,751491534,38137500370290000,1737 HIGHLAND ST MESQUITE TX 75149
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE,75181,60259500050250000,9925 AUTUMN WOOD DR MESQUITE TX 75181
5,38213500050160000,RES,2900,SIDNEY DR,MESQUITE,751502253,38213500050160000,2900 SIDNEY DR MESQUITE TX 75150
9,38139500580020000,RES,1507,RICHARD ST,MESQUITE,751491438,38139500580020000,1507 RICHARD ST MESQUITE TX 75149


In [3]:
addresses = [df['FullAddress']]

In [4]:
# initialize list for successful pulls
houses_list = []

for index, each in enumerate(addresses[0][10000:20000]):

    # reset values
    url = ''
    address = each

    try:
        # attempt to get data from each address
        response = client.search(address)
        url = response['payload']['exactMatch']['url']
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']        

        house_data = client.above_the_fold(property_id, listing_id=None)

        baths = house_data['payload']['addressSectionInfo']['baths']
        beds = house_data['payload']['addressSectionInfo']['beds']
        sq_ft_lot = house_data['payload']['addressSectionInfo']['lotSize']
        sq_ft_interior = house_data['payload']['addressSectionInfo']['sqFt']['value']
        year_built = house_data['payload']['addressSectionInfo']['yearBuilt']
        lat = house_data['payload']['addressSectionInfo']['latLong']['latitude']
        long = house_data['payload']['addressSectionInfo']['latLong']['longitude']
        price = house_data['payload']['addressSectionInfo']['priceInfo']['amount']
        price_label = house_data['payload']['addressSectionInfo']['priceInfo']['label']
        img = initial_info['payload']['preloadImageUrls']

        # add to house_data_df
        house_data = {
            'property_id': property_id,
            'full_address': address,
            'beds': beds,
            'baths': baths,
            'year_built': year_built,
            'sq_ft_interior': sq_ft_interior,
            'sq_ft_lot': sq_ft_lot,
            'price': price,
            'price_label': price_label,
            'lat': lat,
            'long': long,
            'img_url': img
        }

        # append data to successful list here
        houses_list.append(house_data)

    except Exception as e:
        # anything that doesn't have above info gets thrown as error
        print(f'error: {e} at address: {address}')

    if (index+1) % 100 == 0:
            print(f'Processed {index+1} addresses')

error: 'exactMatch' at address: 9922 HICKORY CROSSING ST DALLAS TX 75243
error: 'exactMatch' at address: 9930 HICKORY CROSSING ST DALLAS TX 75243
error: 'exactMatch' at address: 10014 HICKORY CROSSING ST DALLAS TX 75243
error: 'beds' at address: 6108 BLUFF POINT DR DALLAS TX 75248
error: 'beds' at address: 10663 BLACK WALNUT DR DALLAS TX 75243
error: 'beds' at address: 6110 BLACK BERRY LN DALLAS TX 75248
error: 'beds' at address: 6140 YELLOW ROCK TRL DALLAS TX 75248
error: 'exactMatch' at address: 17225 STEDMAN DR DALLAS TX 75252
error: 'exactMatch' at address: 17315 CALLA DR DALLAS TX 75252
Processed 100 addresses
error: 'amount' at address: 3330 SHARPVIEW LN DALLAS TX 75228
error: 'exactMatch' at address: 9926 HICKORY CROSSING ST DALLAS TX 75243
error: 'exactMatch' at address: 10011 HICKORY CROSSING ST DALLAS TX 75243
error: 'exactMatch' at address: 9914 HICKORY CROSSING ST DALLAS TX 75243
Processed 200 addresses
error: 'exactMatch' at address: 9907 HICKORY CROSSING ST DALLAS TX 7524

In [5]:
# create df
house_data_df = pd.DataFrame(
    houses_list
)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url
0,31110507,6214 LA COSA DR DALLAS TX 75248,4,2.5,1979,2661,8041,572377,Redfin Estimate,32.959386,-96.799316,[https://maps.googleapis.com/maps/api/streetvi...
1,31110513,15419 CYPRESS HILLS DR DALLAS TX 75248,6,5.0,1981,4155,9396,714983,Redfin Estimate,32.959420,-96.797048,[https://ssl.cdn-redfin.com/photo/90/mbphoto/0...
2,31110521,6223 BLACK BERRY LN DALLAS TX 75248,4,2.5,1979,2362,8041,607846,Redfin Estimate,32.959071,-96.798638,[https://ssl.cdn-redfin.com/photo/90/mbphoto/5...
3,31110566,15810 RANCHITA DR DALLAS TX 75248,5,4.0,1979,3378,11256,879024,Redfin Estimate,32.963168,-96.802235,[https://maps.googleapis.com/maps/api/streetvi...
4,31110551,6219 SANDYDALE DR DALLAS TX 75248,5,3.5,1979,3273,14201,802826,Redfin Estimate,32.963150,-96.798952,[https://maps.googleapis.com/maps/api/streetvi...


In [6]:
# extract zip code into a new column
house_data_df['zip_code'] = house_data_df['full_address'].str.slice(-5)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url,zip_code
0,31110507,6214 LA COSA DR DALLAS TX 75248,4,2.5,1979,2661,8041,572377,Redfin Estimate,32.959386,-96.799316,[https://maps.googleapis.com/maps/api/streetvi...,75248
1,31110513,15419 CYPRESS HILLS DR DALLAS TX 75248,6,5.0,1981,4155,9396,714983,Redfin Estimate,32.959420,-96.797048,[https://ssl.cdn-redfin.com/photo/90/mbphoto/0...,75248
2,31110521,6223 BLACK BERRY LN DALLAS TX 75248,4,2.5,1979,2362,8041,607846,Redfin Estimate,32.959071,-96.798638,[https://ssl.cdn-redfin.com/photo/90/mbphoto/5...,75248
3,31110566,15810 RANCHITA DR DALLAS TX 75248,5,4.0,1979,3378,11256,879024,Redfin Estimate,32.963168,-96.802235,[https://maps.googleapis.com/maps/api/streetvi...,75248
4,31110551,6219 SANDYDALE DR DALLAS TX 75248,5,3.5,1979,3273,14201,802826,Redfin Estimate,32.963150,-96.798952,[https://maps.googleapis.com/maps/api/streetvi...,75248


In [7]:
house_data_df.value_counts(['price_label'])

price_label    
Redfin Estimate    8430
dtype: int64

In [8]:
house_data_df.dtypes

property_id         int64
full_address       object
beds                int64
baths             float64
year_built          int64
sq_ft_interior      int64
sq_ft_lot           int64
price               int64
price_label        object
lat               float64
long              float64
img_url            object
zip_code           object
dtype: object

In [9]:
house_data_df.shape

(8430, 13)

In [10]:
# export df as csv
house_data_df.to_csv('./redfin_house_data_batch2.csv', index=False)